In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

!pip install six
from sklearn.tree import export_graphviz
from IPython.display import Image  
from six import StringIO
import pydotplus
from sklearn import metrics
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
data=pd.read_csv('chocolate_bars.csv',index_col=0)
data.head(-5)
#print(data)

,manufacturer,company_location,year_reviewed,bean_origin,bar_name,cocoa_percent,num_ingredients,ingredients,review,rating
id,,,,,,,,,,
2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.0,"B,S,C","rich cocoa, fatty, bready",3.25
2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76.0,3.0,"B,S,C","cocoa, vegetal, savory",3.50
2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.0,"B,S,C","cocoa, blackberry, full body",3.75
2542,5150,U.S.A.,2021,Fiji,"Matasawalevu, batch 1",68.0,3.0,"B,S,C","chewy, off, rubbery",3.00
2546,5150,U.S.A.,2021,Venezuela,"Sur del Lago, batch 1",72.0,3.0,"B,S,C","fatty, earthy, moss, nutty,chalky",3.00
...,...,...,...,...,...,...,...,...,...,...
801,Zotter,Austria,2012,Blend,Amazonas Frucht,65.0,4.0,"B,S*,C,Sa","dairy, burnt caramel, coffee",3.50
875,Zotter,Austria,2012,Peru,"Satipo Pangoa region, 20hr conche",70.0,4.0,"B,S*,C,Sa","butterscotch, pepper",3.50
883,Zotter,Austria,2012,Blend,"Indianer, Raw",58.0,4.0,"B,S*,C,Sa","smooth, spicy, floral",3.50


In [33]:
#sns.pairplot(data, hue="id")

In [34]:
#data.isna().sum()

In [35]:
#filling the missing tables with the most common ingredients
data['ingredients'].fillna(data['ingredients'].mode()[0], inplace=True)
data['num_ingredients'].fillna(data['num_ingredients'].mode()[0], inplace=True)
#print(data)

In [36]:
#data.isna().sum()

In [37]:
# Importing LabelEncoder
from sklearn.preprocessing import LabelEncoder
# Instantiating LabelEncoder
le=LabelEncoder()
# Iterating over all the values of each column and extract their dtypes
for col in data.columns.to_numpy():
    # Comparing if the dtype is object
    if data[col].dtypes in ('object','category'):
    # Using LabelEncoder to do the numeric transformation
        data[col]=le.fit_transform(data[col].astype(str))
        
#Binning the rating column
cut_labels = ['really bad', 'bad', 'ok', 'good']
cut_bins = [0, 0.99,1.99,2.99,4.0]
data['rating'] = pd.cut(data['rating'], bins=cut_bins, labels=cut_labels)

x=data.drop("rating",axis=1)
y=data["rating"]
#print(y)
#print(x)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42,stratify=y)
x_train.shape, x_test.shape

'''
#Validation curve
k=np.arange(1,10)
train_score, test_score = validation_curve(DecisionTreeClassifier(), x_train, y_train, param_name='max_depth', param_range=k, scoring="accuracy",cv=5)
plt.plot(k, train_score.mean(axis=1),marker='o', markersize=5,color='blue', label='Training Accuracy')
plt.plot(k, test_score.mean(axis=1),marker='o', markersize=5,color='green', label='Validation Accuracy')
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.grid()
plt.show()
'''
'''
#learning curve

train_size, train_score2, test_score2=learning_curve(DecisionTreeClassifier(max_depth=5,random_state=42,min_samples_split=10,min_samples_leaf=10), x_train, y_train, train_sizes=np.linspace(0.1, 1.0, 10), scoring="accuracy",cv=5)
train_mean = np.mean(train_score2, axis=1)
test_mean = np.mean(test_score2, axis=1)
train_std = np.std(train_score2, axis=1)
test_std = np.std(test_score2, axis=1)
plt.plot(train_size, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(train_size, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_size, test_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
plt.fill_between(train_size, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.title('Learning Curve')
plt.xlabel('Training Data Size')
plt.ylabel('Model accuracy')
plt.grid()
plt.legend(loc='lower right')
plt.show()
'''

import time
startTime = time.time()
##Decision tree
dt=DecisionTreeClassifier(max_depth=5,min_samples_split=10,min_samples_leaf=10,random_state=42)
dt.fit(x_train,y_train)
#Evaluate the accuracy of the model
y_pred = dt.predict(x_test)
predictions = metrics.accuracy_score(y_test, y_pred)
#Calculating the accuracy in percentage
print('The accuracy is: ', predictions * 100, '%')
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

The accuracy is:  78.12911725955203 %
Execution time in seconds: 0.00500178337097168


In [38]:
'''
#vizualising the Tree
new_dot_data = StringIO()
lst=list(x.columns)
for l in lst:
    print(type(l))

print(x.columns)
export_graphviz(dt, out_file=new_dot_data, feature_names=list(x.columns),class_names=y.unique(), filled=True, rounded=True)

graph = pydotplus.graph_from_dot_data(new_dot_data.getvalue())
Image(graph.create_png())
'''

'\n#vizualising the Tree\nnew_dot_data = StringIO()\nlst=list(x.columns)\nfor l in lst:\n    print(type(l))\n\nprint(x.columns)\nexport_graphviz(dt, out_file=new_dot_data, feature_names=list(x.columns),class_names=y.unique(), filled=True, rounded=True)\n\ngraph = pydotplus.graph_from_dot_data(new_dot_data.getvalue())\nImage(graph.create_png())\n'

In [39]:
##calculate the accuracy
from sklearn.metrics import confusion_matrix, accuracy_score
y_train_pred = dt.predict(x_train)
y_test_pred = dt.predict(x_test)

trainaccuracy= accuracy_score(y_train, y_train_pred)
testaccuracy= accuracy_score(y_test, y_test_pred)

confusion_TRN = confusion_matrix(y_train, y_train_pred)
confusion_TST = confusion_matrix(y_test, y_test_pred)

TP = confusion_TRN[1,1] # true positive 
TN = confusion_TRN[0,0] # true negatives
FP = confusion_TRN[0,1] # false positives
FN = confusion_TRN[1,0] # false negatives

TP_TST = confusion_TST[1,1] # true positive 
TN_TST = confusion_TST[0,0] # true negatives
FP_TST = confusion_TST[0,1] # false positives
FN_TST = confusion_TST[1,0] # false negatives

trainsensitivity= TP / float(TP+FN)
trainspecificity= TN / float(TN+FP)

testsensitivity= TP_TST / float(TP_TST+FN_TST)
testspecificity= TN_TST / float(TN_TST+FP_TST)

# Let us compare the values obtained for Train & Test:
print('-'*30)
print('On Train Data')
print('-'*30)
print("Accuracy    : {} %".format(round((trainaccuracy*100),2)))
print("Sensitivity : {} %".format(round((trainsensitivity*100),2)))
print("Specificity : {} %".format(round((trainspecificity*100),2)))
print('-'*30)
print('On Test Data')
print('-'*30)
print("Accuracy    : {} %".format(round((testaccuracy*100),2)))
print("Sensitivity : {} %".format(round((testsensitivity*100),2)))
print("Specificity : {} %".format(round((testspecificity*100),2)))
print('-'*30)


------------------------------
On Train Data
------------------------------
Accuracy    : 79.33 %
Sensitivity : 100.0 %
Specificity : 0.0 %
------------------------------
On Test Data
------------------------------
Accuracy    : 78.13 %
Sensitivity : 100.0 %
Specificity : 0.0 %
------------------------------
